In [1]:
import pandas as pd

In [2]:
filename = 'data/sfpd_clean.csv'
sfdata = pd.read_csv(filename)


In [3]:
from geopy.geocoders import Nominatim
import re
locDict = {}

In [4]:
def get_zipcode(loc):
    geolocator = Nominatim()
    loc = re.sub(r'\(|\)', '', loc)
    location = geolocator.reverse(loc)
    zipcode = location.raw['address']['postcode']
    return zipcode

In [5]:
locations = sfdata['Location'].unique()

In [ ]:
import pyprind
import geopy
import time
import pickle

def get_zip_dict(locations):
    global locDict
    n = len(locations)
    bar = pyprind.ProgBar(n)
    with open('data/sfpost.txt', 'w+') as f:
        for i in xrange(n):
            loc = locations[i]
#             print loc
            if loc not in locDict:
                try:
                    zipcode = get_zipcode(loc)
#                     print zipcode
                except geopy.exc.GeocoderServiceError as e:
                    print e.args[0]
                    if e.args[0] == 'HTTP Error 420: unused':
                        time.sleep(1800)
                        zipcode = get_zipcode(loc) 
                    elif e.args[0] == '<urlopen error [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>':
                        time.sleep(5)
                        zipcode = get_zipcode(loc)
                    elif e.args[0] == '<urlopen error [Errno 65] No route to host>':
                        time.sleep(1800)
                        zipcode = get_zipcode(loc) 
                f.write('{}, {} \n'.format(loc, zipcode))
                locDict[loc] = zipcode
            bar.update()
    return locDict
            
locDict1 = get_zip_dict(locations, locDict)


0%                          100%
[                              ]

<urlopen error [Errno 65] No route to host>


In [29]:
import geocoder
import logging

def get_zip_dict_google(locations):
    locDict = 
    n = len(locations)
    bar = pyprind.ProgBar(n)
    with open('data/sfpost.txt', 'w+') as f:
        for i in xrange(n):
            loc = locations[i]
#             print loc
            if loc not in locDict:
                g = geocoder.google(loc, method='reverse')
                attempts = 1
                while not(g.ok) and attempts < 4:
        #                 logging.warning('Geocoding ERROR: {}'.format(g.debug()))
                    time.sleep(2)  # 2 seconds are specified in the API. If you still get errors, it's because you've reached the daily quota.
                    g = geocoder.google(loc, method='reverse')
                    attempts += 1
                if attempts > 3:
                    logging.warning('Daily quota of google lookups exceeded.')
                    break
                zipcode = g.postal
                f.write('{}, {} \n'.format(loc, zipcode))
                locDict[loc] = zipcode
            bar.update()
    return locDict
locDict1 = get_zip_dict_google(locations, locDict)


0%                          100%
[                              ]

In [30]:
len(locDict)

1067

In [ ]:
import pickle
with open('data/sfpost.pkl', 'wb') as f:
    pickle.dump(locDict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
sfdata['zipcode'] = sfdata['Location'].apply(lambda x: locDict[x])

In [ ]:
sfdata.to_csv(filename)

In [ ]:
sfdata.head()

In [ ]:
sfdf = sfdata.groupby(['Year', 'zipcode','CrimeCat'])['Unnamed: 0'].count()

In [ ]:
sfdf = sfdf.unstack().reset_index()

In [ ]:
sfdf = sfdf.fillna(0)

In [ ]:
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
sfdata['Address'].unique().shape

In [ ]:
sfdf.head()

In [ ]:
X = sfdf.drop(['Year', 'Address'], axis=1).values
year = sfdf['Year'].values

In [ ]:
X.shape

In [ ]:
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

scaler = StandardScaler()
X_trans = scaler.fit_transform(X)
def kmeans_one_year(X, year, yr):
    dists = []
    scores = []
    for k in xrange(2, 10):
        print k
        kmeanModel = KMeans()
        kmeanModel.fit(X[year == yr])
        centroids = kmeanModel.cluster_centers_
        labels = kmeanModel.labels_
        dist = cdist(X, centroids)
        dists.append(dist.min(axis=1).sum())
        score = silhouette_score(X[year == yr], labels)
        scores.append(score)
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(121)
    ax1.plot(xrange(2, 10), dists, 'b*-')
    ax1.grid(True)
    ax1.set_xlabel('Number of clusters')
    ax1.set_ylabel('sum of squares')
    ax2 = fig.add_subplot(122)
    ax2.plot(xrange(2, 50), scores, 'b*-')
    ax2.grid(True)
    ax2.set_xlabel('Number of clusters')
    ax2.set_ylabel('slihouette score')
    plt.show()

kmeans_one_year(X, year, 2003)

In [7]:
sfdata.head()


,Unnamed: 0,Category,Descript,DayOfWeek,PdDistrict,Address,X,Y,Location,CrimeCat,Hour,Month,Day,Year
0,3,ROBBERY,"ROBBERY, BODILY FORCE",4,SOUTHERN,0 Block of UNITEDNATIONS PZ,-122.414318,37.779944,"(37.7799444052046, -122.414317857881)",2,21,6,16,2016
1,6,ASSAULT,ELDER ADULT OR DEPENDENT ABUSE (NOT EMBEZZLEME...,5,NORTHERN,300 Block of FULTON ST,-122.422536,37.778796,"(37.7787958771466, -122.422536478306)",4,12,1,2,2015
2,9,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,5,INGLESIDE,600 Block of LONDON ST,-122.437928,37.719328,"(37.7193276406568, -122.43792838007)",1,0,1,16,2015
3,12,VEHICLE THEFT,STOLEN MOTORCYCLE,5,BAYVIEW,700 Block of SILLIMAN ST,-122.412481,37.729247,"(37.729246748441, -122.412480611343)",5,0,1,16,2015
4,17,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,3,SOUTHERN,0 Block of FOLSOM ST,-122.390556,37.790475,"(37.7904750361645, -122.39055604599)",1,18,1,7,2015
